In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [12]:
# Set the environment variable to point to your service account key
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/rajan/CREDENTIALS/rtc-ai-20240203-c915144c2c55.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/ravi/.config/gcloud/genai-443318-637e44e3cf32.json"

In [13]:
PROJECT_ID = "genai-443318"   # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://reasoning-bucket-1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

### Agent Building 

In [14]:
model = "gemini-1.5-pro-001"

In [15]:
import requests

def get_exchange_rate(
        currency_from: str = "USD",
        currency_to: str = "INR",
        currency_date: str = "latest"
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()


In [16]:
get_exchange_rate(
    currency_from="EUR",
    currency_to="INR",
    currency_date="2012-01-01"
)

{'amount': 1.0, 'base': 'EUR', 'date': '2011-12-30', 'rates': {'INR': 68.713}}

In [17]:
from vertexai.preview import reasoning_engines

In [18]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [19]:
agent.query(input="What's the exchange rate from US dollars to Indian currency today?")

{'input': "What's the exchange rate from US dollars to Indian currency today?",
 'output': 'The exchange rate from US dollars to Indian rupees is 1 USD to 86.7 INR.',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {'currency_from': 'USD', 'currency_to': 'INR'},
     'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'INR'}`\n\n\n",
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{"currency_from": "USD", "currency_to": "INR"}'}},
        'response_metadata': {'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH',
           'probability_label': 'NEGLIGIBLE',
 

In [20]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [21]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "requests",
    ],
)

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/reasoning-bucket-1?projection=noAcl&prettyPrint=false: rag-service@genai-443318.iam.gserviceaccount.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist).

In [ ]:
remote_agent.query(input="What's the exchange rate from US dollars to Indian currency today?")

In [ ]:
remote_agent.resource_name

In [ ]:
REASONING_ENGINE_RESOURCE_NAME = remote_agent.resource_name
remote_agent2 = reasoning_engines.ReasoningEngine(REASONING_ENGINE_RESOURCE_NAME)
remote_agent2.query(input="What's the exchange rate from US dollars to Indian currency today?")

### Model Configuration

In [20]:
## Model variant and version
model = "gemini-1.0-pro-001"

## Model safety settings
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

## Model parameters
model_kwargs = {
    "temperature": 0.3,
    "max_output_tokens": 1000,
    "top_p": 0.95,
    "top_k": 40,
    "safety_settings": safety_settings,
}